This notebook is designed to visualize the GaMMA result, like figures shown in https://github.com/AI4EPS/GaMMA/blob/master/docs/example_synthetic.ipynb

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import Proj
import json
import pygmt

## Configs

In [ ]:
center=(-178, -19)

In [ ]:
with open("../data/stations.json") as json_file:
    stations_raw = json.load(json_file)
stations = pd.DataFrame(columns=["id", "x(km)", "y(km)", "z(km)","longitude","latitude"])
proj = Proj(
    f"+proj=sterea +lon_0={center[0]} +lat_0={center[1]} +units=km")
for key in stations_raw:
    row = stations_raw[key]
    x, y = proj(longitude=row["longitude"], latitude=row["latitude"]) # pylint: disable=unpacking-non-sequence
    z = -row["elevation_m"]/1000
    stations.loc[len(stations)] = [key, x, y, z,row["longitude"],row["latitude"]]

In [ ]:
stations.head()

## load data

In [ ]:
df_global=pd.read_csv('../example_res/picks_global.csv',sep='\t',index_col=0)
df_global = df_global.reset_index(drop=True)
# map event_index to from 0,1,2 to 2,1,0
df_global['event_index']=df_global['event_index'].map({2:0,1:1,0:2})

df_catalog_global=pd.read_csv('../example_res/catalogs_global.csv',sep='\t',index_col=0)
# timestamp to datetime
df_global['timestamp']=pd.to_datetime(df_global['timestamp'])
df_catalog_global['time']=pd.to_datetime(df_catalog_global['time'])
# join df_global and df_catalog_global on event_index
df_global=df_global.join(df_catalog_global.set_index('event_index'),on='event_index',rsuffix='_catalog')
# join df_global and distance in station on id
df_global=df_global.join(stations.set_index('id'),on='id',rsuffix='_station')

df_catalog_local=pd.read_csv('../example_res/catalogs_local.csv',sep='\t',index_col=0)
df_catalog_local['time']=pd.to_datetime(df_catalog_local['time'])

# join df_global and df_catalog_local on event_index, with suffix _local
df_global=df_global.join(df_catalog_local.set_index('event_index'),on='event_index',rsuffix='_local')

In [ ]:
# split df_global with event_index into a list of dataframes
df_global_list = [df_global[df_global['event_index'] == i].copy() for i in df_global['event_index'].unique()]

# for each dataframe in df_global_list, calculate tshift=t_pick-t_catalog
# using .loc[row_indexer,col_indexer] = value syntax
for df in df_global_list:
    df.loc[:, 'tshift'] = df['timestamp'] - df['time']
    # convert tshift to seconds
    df.loc[:, 'tshift'] = df['tshift'].apply(lambda x: x.total_seconds())
    # calculate tshift_local=timestamp-time_local
    df.loc[:, 'tshift_local'] = df['timestamp'] - df['time_local']
    # convert tshift_local to seconds
    df.loc[:, 'tshift_local'] = df['tshift_local'].apply(lambda x: x.total_seconds())

In [ ]:
df_global_list[0].iloc[0]

In [ ]:
ground_truth_mapper={
    0:(pd.to_datetime("2009-12-26T05:15:54.969999"),-20.4003,-175.5445,168.4243),
    2:(pd.to_datetime("2010-06-29T08:16:13.949997"),-17.6654,-177.7512,523.6898),
    1:(pd.to_datetime("2010-05-14T10:33:23.950001"),-20.1968,-177.1223)
}
ground_truth_mapper

In [ ]:
def plot(df,index):
    df = df[df['type'] == 'p']
    df.loc[:, 'tshift'] = df['tshift'] + 20
    
    fig = pygmt.Figure()

    x = 20
    y = df.iloc[0]['longitude']
    x_local = (df.iloc[0]['time_local']-df.iloc[0]['time']).total_seconds()+20
    y_local = df.iloc[0]['longitude_local']
    
    # Create a scatter plot using PyGMT
    fig.plot(
        x=df['tshift'],
        y=df['longitude_station'],
        style='c0.2c',
        fill='blue',
        pen='black',
        projection='X15c/10c',
        region=[0, df['tshift'].max()+20, min(df['longitude_station'].min(),y,y_local)-1, max(df['longitude_station'].max(),y,y_local)+1],
        frame=["WSen","xaf+lTime (s)","yaf+lLongitude (deg)"],
    )
    
    # Plot the event origin location
    fig.plot(x=x, y=y, style="c0.8c", fill=None, pen="3p,red")
    fig.plot(x=x, y=y, style="+0.8c", pen="3p,red")
    
    # Label the station id with non-overlapping
    for i in range(len(df)):
        fig.text(x=df.iloc[i]['tshift'], y=df.iloc[i]['longitude_station'], text=df.iloc[i]['id'].split(".")[1], font='8p', justify='TR')

    ground_truth_x=(ground_truth_mapper[index][0]-df.iloc[0]['time']).total_seconds()+20
    ground_truth_y=ground_truth_mapper[index][2]

    # Plot the ground truth location
    fig.plot(x=ground_truth_x, y=ground_truth_y, style="c0.8c", fill=None, pen="3p,green")
    fig.plot(x=ground_truth_x, y=ground_truth_y, style="+0.8c", pen="3p,green")

    # Plot the local event origin location
    fig.plot(x=x_local, y=y_local, style="c0.8c", fill=None, pen="3p,blue")
    fig.plot(x=x_local, y=y_local, style="+0.8c", pen="3p,blue")

    
    fig.show()

for index in range(3):
    plot(df_global_list[index],index)


In [ ]:
# show the stations' locations on the map using pygmt
def show_map(df,index):
    REGION=[-184, -172, -24, -14]

    fig = pygmt.Figure()
    fig.basemap(region=REGION, projection="M8i",
                        frame=["WSen", "xafg", "yafg"])
    fig.coast(water="white", resolution="l", land="GRAY81",
                borders=["1/0.1p,black"], lakes=["GRAY81"])
    fig.plot(x=stations['longitude'], y=stations['latitude'], style="c0.3c", fill="black")
    # plot longitude and latitude of stations in df in red color
    fig.plot(x=df['longitude_station'], y=df['latitude_station'], style="c0.3c", fill="blue")
    # also text the station id
    for i in range(len(df)):
        fig.text(x=df.iloc[i]['longitude_station'], y=df.iloc[i]['latitude_station'], text=df.iloc[i]['id'].split(".")[1], font='12p', justify='TR',offset="1.5c/0.3c")
    # plot event origin (longitude, latitude) in big circle with no fill
    # print(df.iloc[0]['longitude'], df.iloc[0]['latitude'])
    fig.plot(x=df.iloc[0]['longitude'], y=df.iloc[0]['latitude'], style="c0.8c", fill=None, pen="3p,red")
    fig.plot(x=df.iloc[0]['longitude'], y=df.iloc[0]['latitude'], style="+0.8c", pen="3p,red")
    # fig.text(x=df.iloc[0]['longitude'], y=df.iloc[0]['latitude'], text="INV", font='12p', justify='TR',offset="1.5c/0.3c")
    # print(ground_truth_mapper[index][1],ground_truth_mapper[index][2])
    fig.plot(x=ground_truth_mapper[index][2], y=ground_truth_mapper[index][1], style="c0.8c", fill=None, pen="3p,green")
    fig.plot(x=ground_truth_mapper[index][2], y=ground_truth_mapper[index][1], style="+0.8c", pen="3p,green")
    # fig.text(x=ground_truth_mapper[index][2], y=ground_truth_mapper[index][1], text="GT", font='12p', justify='BL',offset="1.5c/0.3c")

    # plot local event origin (longitude, latitude) in big circle with no fill
    fig.plot(x=df.iloc[0]['longitude_local'], y=df.iloc[0]['latitude_local'], style="c0.8c", fill=None, pen="3p,blue")
    fig.plot(x=df.iloc[0]['longitude_local'], y=df.iloc[0]['latitude_local'], style="+0.8c", pen="3p,blue")

    fig.show()

# show_map(df_global_list[0],0)
for index in range(3):
    show_map(df_global_list[index],index)